In [3]:
%pip install -q -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain import OpenAI, LLMChain
from langchain_openai import OpenAIEmbeddings
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.faiss import FAISS
import os
from dotenv import load_dotenv
import textwrap 
import whisper
import yt_dlp


### Accesing API keys

In [2]:
dotenv_path = '.env'
load_dotenv(dotenv_path)
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['HUGGINGFACEHUB_API_TOKEN'] = os.getenv("HUGGINGFACE_API_TOKEN")

In [ ]:
def download_mp4_from_youtube(url):
  filename="yt_audio.mp4"
  ydl_opts = {
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]',
        'outtmpl': filename,
        'quiet': True,
    }
  with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    result = ydl.extract_info(url, download=True)

def transcribe_video(audio_file):
  model = whisper.load_model("base")
  result = model.transcribe("yt_audio.mp4")
  with open("transcibed_file.txt","w") as file:
    file.write(result['text'])
  

url = "https://www.youtube.com/watch?v=LkhQQOzfczc"
download_mp4_from_youtube(url)
transcribe_video("yt_audio.mp4")

### Calling OpenAI LLM and Embedding models 

In [ ]:
llm = OpenAI(model='text-davinci-003', temperature=0)
embed_model = OpenAIEmbeddings(model="text-embedding-ada-002")

100%|███████████████████████████████████████| 139M/139M [00:01<00:00, 79.9MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 Okay, so one of the biggest challenges as you're building LLM apps is both testing it yourself and getting it in the hands of users to get feedback. So I think there are sort of two main things that you want to think about here. You want to think about building quick UIs that people can test things in, which I'm going to talk about in this video. Then you also want to think about automated testing, which perhaps we saved through a future video. So the first way of doing this is you want to basically get your idea, your app, your product, your model into the hands of end users as quickly as possible. This allows you very quickly to just get feedback on what people like, what people don't like. Often when you're coding and making something for the first time, unless you're making it for yourself, and often even when you are making it for yourself, when you actually start to use it, you realize that a lot of the assumptions you had were perhaps totally wrong. So this is where you're look

### Creating summary from transcribed file

In [ ]:

with open("transcibed_file.txt","r") as file:
  text = file.read()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20, separators = [" ", ",", "\n"])
texts = text_splitter.split_text(text)
docs = [Document(page_content=t) for t in text]
prompt_template = """Find this video transcript:
{text}
Create summary of this video in bullet point\n"""

summary_prompt = PromptTemplate(template=prompt_template,
                        input_variables=["text"])
chain = load_summarize_chain(llm=llm, chain_type="stuff",prompt=summary_prompt)
output_summary = chain.run(docs)
wrapped_text = textwrap.fill(output_summary,
                             width=1000,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)